In [ ]:
import pandas as pd
from tqdm import tqdm
import vectorbt as vbt
from helpers.vietcap import VietCapClient

### DANH SÁCH MÃ CP THUỘC NHÓM VN200
> VN200 là top 200 mã CP có `ev` (*vốn hóa*) lớn nhất

In [2]:
vcsc = VietCapClient()
# tất cả các mã CP đang hoạt động
df_stocks = vcsc.get_stocks()
df_stocks.shape

(1586, 25)

In [3]:
# top 200 mã CP có vốn hóa lớn nhất
df_vn200 = (
    df_stocks
    .sort_values("ev", ascending=False)
    .head(200)
    .sort_values("ticker")
    .reset_index(drop=True)
    .loc[:,["ticker","exchange","closePrice","totalVolume","ev","pe","pb","roa","roe"]]
)
df_vn200

,ticker,exchange,closePrice,totalVolume,ev,pe,pb,roa,roe
0,ABB,UPCOM,13000,9058594,1.341408e+13,9.217371,0.875189,0.007979,0.102281
1,ACB,HOSE,28450,42851281,1.461379e+14,8.608913,1.675689,0.019587,0.201725
2,ACG,HOSE,37150,40195,5.601772e+12,12.634534,1.318014,0.078590,0.104978
3,ACV,UPCOM,63700,650670,2.283158e+14,23.172810,3.569742,0.127202,0.161737
4,AGR,HOSE,19050,3983370,4.349339e+12,36.617936,1.759683,0.037223,0.048503
...,...,...,...,...,...,...,...,...,...
195,VRE,HOSE,29950,8366915,6.805594e+13,15.458858,1.535110,0.079112,0.103469
196,VSC,HOSE,35000,23363272,1.310296e+13,30.487361,2.809125,0.052375,0.092487
197,VSF,UPCOM,27100,20171,1.356750e+13,3052.825406,5.941906,0.000554,0.001944
198,VSH,HOSE,47950,2234,1.132777e+13,13.904289,2.268005,0.094741,0.169274


### LỊCH SỬ GIAO DỊCH
***Lấy dữ liệu lịch sử giao dịch của các mã CP thuộc nhóm VN200***

In [4]:
vcsc = VietCapClient()
# lấy dữ liệu lịch sử từ 2016
from_date = "2016-01-01"
df_historical_price = pd.DataFrame()
for ticker in tqdm(df_vn200["ticker"]):
    # lấy dữ liệu lịch sử từ vcsc
    data = vcsc.get_historical_price(ticker, from_date=from_date)
    # chỉ append nếu có dữ liệu
    if len(data) > 0:
        # chuyển dữ liệu dạng list thành bảng
        df = pd.DataFrame(data)
        # chuyển cột `tradingDate` từ dạng timestamp thành datetime
        df["tradingDate"] = pd.to_datetime(df["tradingDate"], unit="ms")
        # append dữ liệu
        df_historical_price = pd.concat([df_historical_price, df], ignore_index=True)

100%|██████████| 200/200 [00:39<00:00,  5.10it/s]


In [5]:
# dữ liệu giao dịch có nhiều cột
df_historical_price.iloc[0]

tradingDate                 2025-08-19 00:00:00
ticker                                      ABB
openPrice                                 13000
highestPrice                              13200
lowestPrice                               12800
closePrice                                13000
openPriceAdjusted                       13000.0
highestPriceAdjusted                    13200.0
lowestPriceAdjusted                     12800.0
closePriceAdjusted                      12960.0
totalMatchVolume                        9058593
totalMatchValue                    117407363700
totalDealVolume                               1
totalDealValue                            11100
unMatchedBuyTradeVolume               6597857.0
unMatchedSellTradeVolume              9712826.0
totalVolume                             9058594
totalValue                         117407374800
totalBuyTrade                            4141.0
totalBuyTradeVolume                  15656450.0
totalSellTrade                          

In [6]:
# rút ngắn dữ liệu cần thiết cho BACKTESTING gồm giá `OHLC` (điều chỉnh) và KL khớp lệnh `totalMatchVolume`
# tính thêm các chỉ số cần thiết để BACKTESTING
df_historical_price_bt = (
    df_historical_price
    .loc[:,["tradingDate","ticker","closePriceAdjusted"]]
    .rename(columns=lambda x: x.replace("PriceAdjusted", ""))
    .sort_values(["ticker","tradingDate"])
    .reset_index(drop=True)
    .assign(
        SMA10  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(10).mean()),
        SMA20  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(20).mean()),
        SMA50  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(50).mean()),
        SMA100 = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(100).mean()),
        EMA12  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.ewm(span=12, adjust=False).mean()),
        EMA26  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.ewm(span=26, adjust=False).mean()),
        MACD   = lambda df: df["EMA12"] - df["EMA26"],
        MACDSignal = lambda df: df.groupby("ticker")["MACD"].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    )
)
df_historical_price_bt.head()

,tradingDate,ticker,close,SMA10,SMA20,SMA50,SMA100,EMA12,EMA26,MACD,MACDSignal
0,2020-12-28,ABB,13910.0,NaN,NaN,NaN,NaN,13910.000000,13910.000000,0.000000,0.000000
1,2020-12-29,ABB,13737.0,NaN,NaN,NaN,NaN,13883.384615,13897.185185,-13.800570,-2.760114
2,2020-12-30,ABB,13620.0,NaN,NaN,NaN,NaN,13842.863905,13876.652949,-33.789044,-8.965900
3,2020-12-31,ABB,13447.0,NaN,NaN,NaN,NaN,13781.961766,13844.826805,-62.865039,-19.745728
4,2021-01-04,ABB,13341.0,NaN,NaN,NaN,NaN,13714.121494,13807.506301,-93.384806,-34.473543


In [7]:
# xây dựng logic tín hiệu MUA
## đặt biến
sma10       = df_historical_price_bt["SMA10"]
sma20       = df_historical_price_bt["SMA20"]
sma50       = df_historical_price_bt["SMA50"]
sma100      = df_historical_price_bt["SMA100"]
macd        = df_historical_price_bt["MACD"]
macd_signal = df_historical_price_bt["MACDSignal"]
sma10_lag1  = df_historical_price_bt.groupby("ticker")["SMA10"].shift(1)
sma20_lag1  = df_historical_price_bt.groupby("ticker")["SMA20"].shift(1)
## SMA10 cắt lên SMA20
sma10_crossover_sma20 = (sma10 > sma20) & (sma10_lag1 <= sma20_lag1)
## SMA20 > SMA50
sma20_over_sma_50 = sma20 > sma50
## SMA50 > SMA100
sma50_over_sma_100 = sma50 > sma100
## MACD > MACDsignal
macd_over_macdsignal = macd > macd_signal
## tín hiệu MUA
buy_signal = sma10_crossover_sma20 & sma20_over_sma_50 & sma50_over_sma_100 & macd_over_macdsignal

In [8]:
# xây dựng logic tín hiệu BÁN
## SMA10 cắt xuống SMA20
sell_signal = (sma10 < sma20) & (sma10_lag1 >= sma20_lag1)

In [9]:
## gán ngược tín hiệu MUA/BÁN vào bảng
df_historical_price_bt["buySignal"]  = buy_signal
df_historical_price_bt["sellSignal"] = sell_signal
df_historical_price_bt.head()

,tradingDate,ticker,close,SMA10,SMA20,SMA50,SMA100,EMA12,EMA26,MACD,MACDSignal,buySignal,sellSignal
0,2020-12-28,ABB,13910.0,NaN,NaN,NaN,NaN,13910.000000,13910.000000,0.000000,0.000000,False,False
1,2020-12-29,ABB,13737.0,NaN,NaN,NaN,NaN,13883.384615,13897.185185,-13.800570,-2.760114,False,False
2,2020-12-30,ABB,13620.0,NaN,NaN,NaN,NaN,13842.863905,13876.652949,-33.789044,-8.965900,False,False
3,2020-12-31,ABB,13447.0,NaN,NaN,NaN,NaN,13781.961766,13844.826805,-62.865039,-19.745728,False,False
4,2021-01-04,ABB,13341.0,NaN,NaN,NaN,NaN,13714.121494,13807.506301,-93.384806,-34.473543,False,False


### BACKTESTING

In [ ]:
backtesting_results = {}
for ticker, df in tqdm(df_historical_price_bt.groupby("ticker")):
    # dùng cột ngày để làm index cho bảng
    df.set_index("tradingDate", inplace=True)
    # loại bỏ 100 phiên đầu tiên vì tính `SMA100`
    df = df.iloc[99:]
    if len(df) > 100: # chỉ backtesting đối với mã CP từ 200 phiên giao dịch trở lên
        # chạy backtesting bằng `vectorbt`
        pf = vbt.Portfolio.from_signals(
            close=df["close"],
            entries=df["buySignal"],
            exits=df["sellSignal"],
            size=1.0,       # mua toàn bộ vốn cho mỗi lệnh
            fees=0.001,     # phí giao dịch 0.1%
            freq="1D"
        )
        # lưu kết quả backtesting
        backtesting_results[ticker] = pf

100%|██████████| 200/200 [00:04<00:00, 43.55it/s] 


In [52]:
# chuyển kết quả backtesting thành bảng
df_backtesting = (
    pd.concat({ticker: pf.stats() for ticker, pf in backtesting_results.items()}, axis=1)
    .transpose()
)

In [58]:
# lưu kết quả backtesting
end_date = df_backtesting["End"].max().strftime("%Y%m%d")
df_backtesting.to_csv("./results/backtesting_results_vn200_{}.csv".format(end_date))

In [63]:
# các thông tin cần thiết để phân tích kết quả backtesting
df_backtesting_sf = (
    df_backtesting
    .loc[
        :,
        [
            "Start","End","Period", # dữ liệu lịch sử dùng để backtesting
            "Start Value","End Value","Total Return [%]", # đánh giá lợi nhuận
            "Max Drawdown [%]", # đánh giá rủi ro
            "Profit Factor", # đánh giá hiệu quả giao dịch
            "Sharpe Ratio", # đánh giá hiệu quả điều chỉnh rủi ro
        ]
    ]
    .assign(
        Start = lambda df: df["Start"].apply(lambda x: x.date()),
        End = lambda df: df["End"].apply(lambda x: x.date()),
        Period = lambda df: df["Period"].apply(lambda x: pd.Timedelta(days=x.days))
    )
    .map(lambda x: round(x,2) if isinstance(x, float) else x)
)
df_backtesting_sf

,Start,End,Period,Start Value,End Value,Total Return [%],Max Drawdown [%],Profit Factor,Sharpe Ratio
ABB,2021-05-27,2025-08-19,1057 days,100.0,88.79,-11.21,12.65,0.00,-0.98
ACB,2016-06-06,2025-08-19,2299 days,100.0,103.62,3.62,38.86,1.07,0.12
ACG,2021-12-23,2025-08-19,902 days,100.0,97.21,-2.79,2.88,0.00,-0.72
ACV,2017-04-18,2025-08-19,2085 days,100.0,271.01,171.01,15.05,12.14,1.34
AGR,2016-06-06,2025-08-19,2303 days,100.0,100.69,0.69,51.17,1.01,0.11
...,...,...,...,...,...,...,...,...,...
VRE,2018-04-02,2025-08-19,1845 days,100.0,100.35,0.35,19.27,1.02,0.05
VSC,2016-06-06,2025-08-19,2303 days,100.0,63.87,-36.13,38.45,0.01,-0.76
VSF,2018-09-13,2025-08-19,1731 days,100.0,75.95,-24.05,47.29,0.29,-0.19
VSH,2016-06-06,2025-08-19,2304 days,100.0,100.80,0.80,38.26,1.01,0.10


### KẾT QUẢ PHÂN TÍCH 
***Nhờ chatGPT đọc kết quả backtesting :)))***

***0. Định nghĩa các chỉ số quan trọng***
- `Total Return [%]`: Tổng lời/lỗ của chiến lược trên mã đó trong giai đoạn test → *Ví dụ*: 120% nghĩa là vốn 100 triệu → cuối cùng thành 220 triệu.
- `Max Drawdown [%]`: Mức sụt giảm lớn nhất từ đỉnh về đáy → Giống như khi đang có lãi nhưng sau đó cổ phiếu rớt mạnh. Nếu Max DD = –40%, tức là có lúc tài khoản mất gần một nửa.
- `Sharpe Ratio`: Đo “lãi trên rủi ro” → Số càng cao thì càng “đáng đồng tiền bát gạo”. >1 coi như ổn, >2 là rất tốt.
- `Profit Factor (PF)`: Tổng lãi / Tổng lỗ → *Ví dụ*, PF = 2 nghĩa là kiếm được gấp đôi số tiền đã thua lỗ. PF < 1 thì hệ thống đang lỗ nhiều hơn lãi.

***1. Rule chấm điểm tổng thể***
- `Total Return [%]`:
    ```python
        if   r["Total Return [%]"] >= 100: score += 2 # thêm 2 điểm
        elif r["Total Return [%]"] >= 50 : score += 1 # thêm 1 điểm
        elif r["Total Return [%]"] <= -20: score -= 1 # trừ  1 điểm
        elif r["Total Return [%]"] <= -40: score -= 2 # trừ  2 điểm
    ```
- `Sharpe Ratio`:
    ```python
        if   r["Sharpe Ratio"] >= 1.0  : score += 2 # thêm 2 điểm
        elif r["Sharpe Ratio"] >= 0.5  : score += 1 # thêm 1 điểm
        elif r["Sharpe Ratio"] <  0    : score -= 1 # trừ  1 điểm
    ```
- `Max Drawdown [%]`:
    ```python
        if   r["Max Drawdown [%]"] <= 20: score += 2 # thêm 2 điểm
        elif r["Max Drawdown [%]"] <= 35: score += 1 # thêm 1 điểm
        elif r["Max Drawdown [%]"] >= 60: score -= 1 # trừ  1 điểm
    ```
- `Profit Factor (PF)`:
    ```python
        if r["Profit Factor (PF)"]   >= 2: score += 1 # thêm 1 điểm
        elif r["Profit Factor (PF)"] <  1: score -= 1 # trừ  1 điểm
    ```

***2. Phân loại chất lượng***
- **Strong**: score > 3
- **Good**: 1 < score <= 3
- **Neutral**: 0 <= score <= 1
- **Weak**: score < 0

***3. Kết luận***
- **Chọn lọc** nhóm **Strong**, lợi nhuận cao, rủi ro thấp, Sharpe tốt
- **Bỏ qua** nhóm **Weak**, lỗ nhiều, rủi ro cao
- **Theo dõi** nhóm **Neutral** để test thêm dữ liệu

In [ ]:
def bucket_row(r):
    """Hàm chấm điểm tổng thể"""
    score = 0
    # Return
    tr = r["Total Return [%]"]
    if   tr >= 100: score += 2
    elif tr >=  50: score += 1
    elif tr <= -20: score -= 1
    elif tr <= -40: score -= 2
    # Sharpe
    sr = r["Sharpe Ratio"]
    if pd.notna(sr):
        if   sr >= 1.0: score += 2
        elif sr >= 0.5: score += 1
        elif sr <  0  : score -= 1
    # MaxDD (lower better)
    dd = r["Max Drawdown [%]"]
    if pd.notna(dd):
        if   dd <= 20: score += 2
        elif dd <= 35: score += 1
        elif dd >= 60: score -= 1
    # Profit factor
    pf = r["Profit Factor"]
    if pd.notna(pf):
        if   pf >= 2: score += 1
        elif pf <  1: score -= 1
    return score

In [71]:
# tính toán điểm và phân loại cho tất cả các mã CP
df_backtesting_sf["Quality Score"] = df_backtesting_sf.apply(bucket_row, axis=1)
df_backtesting_sf["Bucket"] = pd.cut(
    df_backtesting_sf["Quality Score"],
    bins=[-10,-1,1,3,100],
    labels=["Weak","Neutral","Good","Strong"]
)
df_backtesting_sf["Bucket"].value_counts().sort_index()

Bucket
Weak       68
Neutral    58
Good       35
Strong     36
Name: count, dtype: int64

In [85]:
# các mã CP triển vọng
## STRONG
## Total Return [%] >= 50
## Sharpe Ratio >= 0.5
## Max Drawdown [%] <= 35
## Profit Factor >= 2
df_backtesting_potentials = df_backtesting_sf[
    (df_backtesting_sf["Bucket"]=="Strong") &
    (df_backtesting_sf["Total Return [%]"] >= 50) &
    (df_backtesting_sf["Sharpe Ratio"] >= 0.5) &
    (df_backtesting_sf["Max Drawdown [%]"] <= 35) &
    (df_backtesting_sf["Profit Factor"] >= 2)
]
print("Số mã CP triển vọng:", len(df_backtesting_potentials))
print("Các mã CP triển vọng:", ", ".join(df_backtesting_potentials.index.tolist()))

Số mã CP triển vọng: 24
Các mã CP triển vọng: ACV, ANV, BCM, BIC, CTD, CTR, DHT, GEG, GVR, HUT, KDC, MBB, ORS, PDR, SHS, SSI, STB, TMS, VCR, VGI, VIB, VIC, VND, VPB


In [96]:
# các mã CP phải bỏ qua
## WEAK
## Total Return [%] <= -20
## Sharpe Ratio < 0
## Max Drawdown [%] >= 60
## Profit Factor < 1
df_backtesting_ignored = df_backtesting_sf[
    (df_backtesting_sf["Bucket"]=="Weak") &
    (df_backtesting_sf["Total Return [%]"] <= -20) &
    (df_backtesting_sf["Sharpe Ratio"] < 0) &
    (df_backtesting_sf["Max Drawdown [%]"] >= 60) &
    (df_backtesting_sf["Profit Factor"] < 1)
]
print("Số mã CP bỏ qua:", len(df_backtesting_ignored))
print("Các mã CP bỏ qua:", ", ".join(df_backtesting_ignored.index.tolist()))

Số mã CP bỏ qua: 6
Các mã CP bỏ qua: DSC, HHV, MPC, PDN, SJG, TCB


In [99]:
# các mã CP theo dõi thêm
## NEUTRAL
df_backtesting_watchlist = df_backtesting_sf[
    (df_backtesting_sf["Bucket"]=="Neutral")
]
print("Số mã CP theo dõi:", len(df_backtesting_watchlist))
print("Các mã CP theo dõi:", ", ".join(df_backtesting_watchlist.index.tolist()))

Số mã CP theo dõi: 58
Các mã CP theo dõi: ABB, ACB, ACG, AGR, BID, BSR, BVH, CC1, CHP, CMG, CTG, DBD, DPM, DXG, FTS, HAG, HCM, HDC, HHS, HPG, IDC, IDP, IMP, KBC, KLB, KSF, LPB, MSB, MWG, NKG, NVB, NVL, PAN, PAP, PGV, POW, SAS, SCG, SCS, SGB, SJS, SSB, SSH, TID, TIN, TLG, TMP, VAB, VBB, VCS, VDS, VEA, VEF, VHM, VIF, VJC, VSH, VTP


In [ ]:
# biểu diễn biểu đồ vốn của 1 mã triển vọng
ignored_ticker = df_backtesting_potentials.index[0]
pf = backtesting_results[ignored_ticker]
print(ignored_ticker)
pf.plot().show()

ACV


In [95]:
# biểu diễn biểu đồ vốn của 1 mã bỏ qua
ignored_ticker = df_backtesting_ignored.index[0]
pf = backtesting_results[ignored_ticker]
print(ignored_ticker)
pf.plot().show()

DSC


In [ ]:
# print(df_backtesting_sf.head(5).map(lambda x: round(x,2) if isinstance(x, float) else x).to_markdown())